In [241]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

In [256]:
class GreaterDataset(Dataset):
  def __init__(self, count):
    super(GreaterDataset, self).__init__()

    self.values = [torch.rand((2,)) for i in range(count)]
    self.labels = [torch.tensor([1.]) if x > y else torch.tensor([0.]) for (x, y) in self.values]

  def __len__(self):
    return len(self.values)  

  def __getitem__(self, index):
    return self.values[index], self.labels[index]

In [257]:
points = 10000
train_data = GreaterDataset(points)
test_data = GreaterDataset(points)

train_dataloader = DataLoader(train_data, batch_size = 16)
test_dataloader = DataLoader(test_data, batch_size = 16)

In [279]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.linear(x)
        return x

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=2, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [280]:
loss_fn = nn.BCELoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [281]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [310]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (torch.where(pred > 0.5, 1., 0.) == y).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [311]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 0.211591  [    0/10000]
loss: 0.256141  [ 1600/10000]
loss: 0.243782  [ 3200/10000]
loss: 0.246749  [ 4800/10000]
loss: 0.298765  [ 6400/10000]
loss: 0.321668  [ 8000/10000]
loss: 0.260693  [ 9600/10000]
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.016929 

Epoch 2
-------------------------------
loss: 0.211099  [    0/10000]
loss: 0.255562  [ 1600/10000]
loss: 0.243202  [ 3200/10000]
loss: 0.246195  [ 4800/10000]
loss: 0.298206  [ 6400/10000]
loss: 0.321118  [ 8000/10000]
loss: 0.260243  [ 9600/10000]
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.016895 

Epoch 3
-------------------------------
loss: 0.210612  [    0/10000]
loss: 0.254986  [ 1600/10000]
loss: 0.242626  [ 3200/10000]
loss: 0.245645  [ 4800/10000]
loss: 0.297651  [ 6400/10000]
loss: 0.320572  [ 8000/10000]
loss: 0.259797  [ 9600/10000]
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.016861 

Epoch 4
-------------------------------
loss: 0.210128  [    0/10000]
loss: 0.254416  [ 1600/1

In [308]:
model(torch.tensor([0.911, 0.9]))

tensor([0.5008], grad_fn=<SigmoidBackward0>)

In [306]:
model.state_dict()

OrderedDict([('linear.0.weight', tensor([[ 4.4675, -4.4877]])),
             ('linear.0.bias', tensor([-0.0309]))])

tensor(2)